In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from scipy import stats
from scipy.io import loadmat
from sklearn.cluster.bicluster import SpectralBiclustering, SpectralCoclustering

import cPickle
from eegtools.io import load_edf

In [2]:
band1 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract1.dumps'))
band2 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract2.dumps'))
band3 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract3.dumps'))
band4 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract4.dumps'))
band5 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract5.dumps'))
band6 = np.transpose(np.load('C:\Users\Fioger\Desktop\eeg-viz/MedianSubtract6.dumps'))
print len(band1), len(band1[0])

600000 61


In [4]:
data = np.transpose(np.concatenate((band1, band2, band3, band4, band5, band6)))
print len(data), len(data[0])

61 3383766


In [5]:
length_of_intervals = 30000
channels = 61
edges = 1891
total_interval = len(data[0])/length_of_intervals + 1
channels_data = [[None for j in range(edges)] for i in range(total_interval)] 

In [6]:
def calculate_pearson(interval):
    index_counter = 0
    j = length_of_intervals * interval
    for start in range(channels):
        for index in range(channels):
            if start <= index:
                if interval != total_interval:
                    channels_data[interval][index_counter] = pearsonr(data[start][j:j + length_of_intervals],
                                                                     data[index][j:j + length_of_intervals])[0]
                else:
                    channels_data[interval][index_counter] = pearsonr(data[start][j:], data[index][j:])[0]
                index_counter += 1
    print index_counter

In [7]:
for i in range(total_interval):
    print 'Calculating Interval %s' % i,
    calculate_pearson(i)

Calculating Interval 0 1891
Calculating Interval 1 1891
Calculating Interval 2 1891
Calculating Interval 3 1891
Calculating Interval 4 1891
Calculating Interval 5 1891
Calculating Interval 6 1891
Calculating Interval 7 1891
Calculating Interval 8 1891
Calculating Interval 9 1891
Calculating Interval 10 1891
Calculating Interval 11 1891
Calculating Interval 12 1891
Calculating Interval 13 1891
Calculating Interval 14 1891
Calculating Interval 15 1891
Calculating Interval 16 1891
Calculating Interval 17 1891
Calculating Interval 18 1891
Calculating Interval 19 1891
Calculating Interval 20 1891
Calculating Interval 21 1891
Calculating Interval 22 1891
Calculating Interval 23 1891
Calculating Interval 24 1891
Calculating Interval 25 1891
Calculating Interval 26 1891
Calculating Interval 27 1891
Calculating Interval 28 1891
Calculating Interval 29 1891
Calculating Interval 30 1891
Calculating Interval 31 1891
Calculating Interval 32 1891
Calculating Interval 33 1891
Calculating Interval 34 

In [8]:
post_process_data = np.array(channels_data)
spectral_model = SpectralBiclustering()
spectral_model.fit(post_process_data)
fit_data = post_process_data[np.argsort(spectral_model.row_labels_)]
fit_data = fit_data[:, np.argsort(spectral_model.column_labels_)]
plt.matshow(post_process_data, cmap=plt.cm.Blues)
plt.title('Original Pearson after Median Filter 30s intervals\n')
plt.show()
plt.matshow(fit_data, cmap=plt.cm.Blues)
plt.title('Biclustered Pearson after Median Filter 30s intervals\n')
plt.show()

In [9]:
import json as simplejson
with open('C:\Users\Fioger\Desktop\eeg-viz\postprocess/pearson/median_filter/median-filter-matrix-30s.json', 'w+') as f:
    p = [[float(column) for column in row] for row in post_process_data]
    f.write(simplejson.dumps({'name': 's5d2nap', 'data': p}))

cols = np.argsort(spectral_model.column_labels_)
rows = np.argsort(spectral_model.row_labels_)
indices_matrix = []   
for row in rows:
    for col in cols:
        indices_matrix.append((float(row),float(col)))

with open('C:\Users\Fioger\Desktop\eeg-viz\postprocess/pearson/median_filter/fitted-indices-median-filter-pearson-30s.json', 'w+') as f:
    f.write(simplejson.dumps({'name': 's5d2nap', 'data': indices_matrix}))

# with open('C:\Users\Fioger\Desktop\eeg-viz\postprocess/pearson/z_score/z_score_pearson_30s_dump.dump', 'wb') as f:
#     cPickle.dump(channels_data, f)